# PDF Extraction Report for Club Blackout
This notebook extracts text and styling rules from the Club Blackout Host and Player guides.

In [ ]:
# 1. Install and Import Dependencies
import sys
!{sys.executable} -m pip install pypdf
import pypdf
import re
import json

In [ ]:
# 2. Define File Paths and Initialize PDF Objects
host_guide_path = r"C:\Users\kimro\Downloads\Club Blackout (Booklet).pdf"
player_guide_path = r"C:\Users\kimro\Downloads\Club Blackout Player Booklet A5 (1).pdf"

host_reader = pypdf.PdfReader(host_guide_path)
player_reader = pypdf.PdfReader(player_guide_path)

In [ ]:
# 3. Define Text Extraction and Section Filtering Logic
def extract_text_from_reader(reader):
    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() + "\n"
    return full_text

def get_section(text, section_name, next_sections=[]):
    pattern = rf"(?i){section_name}.*?\n(.*?)(?:\n(?:" + "|".join(next_sections) + r")|$)"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return "Section not found."

host_text = extract_text_from_reader(host_reader)
player_text = extract_text_from_reader(player_reader)

In [ ]:
# 4. Extract Host Guide Content
host_sections = {
    "Overview": get_section(host_text, "Overview", ["Setup", "Night Phase"]),
    "Setup": get_section(host_text, "Setup", ["Overview", "Night Phase"]),
    "Night Phase": get_section(host_text, "Night Phase", ["Day Phase", "Overview"]),
    "Day Phase": get_section(host_text, "Day Phase", ["Roles", "Overview"])
}

for section, content in host_sections.items():
    print(f"--- {section} ---")
    print(content[:500] + "..." if len(content) > 500 else content)
    print("\n")

In [ ]:
# 5. Extract Player Guide Content (Roles and Mechanics)
player_roles = get_section(player_text, "Roles", ["Mechanics", "Overview"])
print("--- Roles ---")
print(player_roles[:1000] + "..." if len(player_roles) > 1000 else player_roles)

In [ ]:
# 6. Extract Metadata and Styling Elements
print("--- Host Guide Metadata ---")
print(json.dumps(dict(host_reader.metadata), indent=4))

print("\n--- Player Guide Metadata ---")
print(json.dumps(dict(player_reader.metadata), indent=4))

# Attempt to find font information
def get_fonts(reader):
    fonts = set()
    for page in reader.pages:
        if "/Resources" in page and "/Font" in page["/Resources"]:
            f = page["/Resources"]["/Font"]
            for key in f:
                fonts.add(f[key]["/BaseFont"])
    return fonts

print("\n--- Host Guide Fonts ---")
print(get_fonts(host_reader))

print("\n--- Player Guide Fonts ---")
print(get_fonts(player_reader))

# 7. Summary of Findings
The extracted text and metadata are displayed above. Use this information to update the game logic and UI styles.